In [ ]:
%pip install onnxruntime opencv-python numpy

In [ ]:
import sys
import cv2
import numpy as np
import onnxruntime as ort

def load_labels(path):
    """Each line in `path` is one class name (zero‑based index)."""
    with open(path, 'r') as f:
        return [l.strip() for l in f if l.strip()]

def preprocess(img: np.ndarray, input_shape):
    """
    Resize & normalize to [0,1], convert BGR→RGB, 
    HWC → CHW, add batch dim.
    """
    _, c, h, w = input_shape  # e.g. [1,3,224,224]
    img_resized = cv2.resize(img, (w, h))
    img_rgb     = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    chw         = np.transpose(img_rgb, (2, 0, 1))
    return np.expand_dims(chw, axis=0)  # [1,3,h,w]

def softmax(x: np.ndarray):
    e = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e / e.sum(axis=1, keepdims=True)

def main(model_path, image_path, labels_path):
    # Load class names:
    # labels = load_labels(labels_path)

    # Create ONNX runtime session:
    sess = ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])

    # Grab I/O metadata:
    input_meta = sess.get_inputs()[0]
    input_name = input_meta.name
    input_shape = input_meta.shape  # [batch,channel,height,width]

    # Load & preprocess image:
    img = cv2.imread(image_path)
    if img is None:
        print(f"ERROR: cannot read '{image_path}'")
        sys.exit(1)
    input_tensor = preprocess(img, input_shape)

    # Run inference:
    raw_outputs = sess.run(None, {input_name: input_tensor})
    logits      = raw_outputs[0]            # shape [1, NC]
    probs       = softmax(logits)           # shape [1, NC]

    # Pick top‑1:
    idx  = int(np.argmax(probs, axis=1)[0])
    prob = float(probs[0, idx])

    # Print and show:
    print(f"→ Predicted: {idx} ({prob*100:.2f}%)")
    text = f"{idx}: {prob*100:.1f}%"
    cv2.putText(img, text, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imshow("Classification", img)
    cv2.waitKey(0)

if __name__ == "__main__":
    if len(sys.argv) != 4:
        print("Usage: python classify.py <model.onnx> <image.jpg> <classes.names>")
        sys.exit(1)
    main(sys.argv[1], sys.argv[2], sys.argv[3])


In [ ]:
import cv2
import numpy as np
import onnxruntime as ort
import string

def load_labels():
    """Create labels for ASL alphabet"""
    labels = []
    # Add uppercase letters A-Z
    labels.extend(list(string.ascii_uppercase))
    # Add special classes
    labels.extend(["del", "nothing", "space"])
    return labels

def preprocess(img: np.ndarray, input_shape):
    """
    Resize & normalize to [0,1], convert BGR→RGB, 
    HWC → CHW, add batch dim.
    """
    _, c, h, w = input_shape  # e.g. [1,3,224,224]
    img_resized = cv2.resize(img, (w, h))
    img_rgb     = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    chw         = np.transpose(img_rgb, (2, 0, 1))
    return np.expand_dims(chw, axis=0)  # [1,3,h,w]

def softmax(x: np.ndarray):
    e = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e / e.sum(axis=1, keepdims=True)

def main():
    # Model path - adjust this to your model location
    model_path = "./models/best.onnx"
    
    # Load class names
    labels = load_labels()
    
    # Create ONNX runtime session
    print("Loading ONNX model...")
    sess = ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])
    
    # Grab I/O metadata
    input_meta = sess.get_inputs()[0]
    input_name = input_meta.name
    input_shape = input_meta.shape  # [batch,channel,height,width]
    print(f"Model expects input shape: {input_shape}")
    
    # Open webcam
    print("Opening webcam...")
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return
    
    print("Press 'q' to quit")
    
    while True:
        # Capture frame
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame")
            break
        
        # Make a copy for display
        display_frame = frame.copy()
        
        # Preprocess frame
        input_tensor = preprocess(frame, input_shape)
        
        # Run inference
        raw_outputs = sess.run(None, {input_name: input_tensor})
        logits = raw_outputs[0]            # shape [1, NC]
        probs = softmax(logits)           # shape [1, NC]
        
        # Pick top‑3 predictions
        top_indices = np.argsort(probs[0])[::-1][:3]
        
        # Display results on frame
        y_offset = 30
        for i, idx in enumerate(top_indices):
            prob = float(probs[0, idx])
            if prob > 0.05:  # Only show predictions with >5% confidence
                label_text = labels[idx] if idx < len(labels) else f"Class {idx}"
                text = f"{label_text}: {prob*100:.1f}%"
                # Position text with increasing y offset for each prediction
                cv2.putText(display_frame, text, (10, y_offset), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                y_offset += 30
        
        # Show frame
        cv2.imshow("ASL Detection", display_frame)
        
        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release resources
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()